In [63]:
%matplotlib inline
from fenparsev4 import *
from pybrain.datasets import ClassificationDataSet
from pybrain.supervised.trainers import BackpropTrainer
from pybrain.tools.shortcuts import buildNetwork
from pybrain.structure.modules import TanhLayer
import numpy as np
import random
from sklearn.metrics import confusion_matrix
import os

In [64]:
#most important part
def fries_ready():
    os.system('say your fries are done')

#returns dataframe with certain columns converted back to lists/integers
def unpickle_csv(filename_prefix='../data/csv_input/2015-12-07-csv', filename_suffix_range=range(1,21), debug=True):
    #read in csv
    df = pd.DataFrame()
    for i in filename_suffix_range:
        if debug: print(filename_prefix + str(i))
        df = df.append(pd.read_csv(filename_prefix + str(i)))
    
    #clean columns
    df['y'] = df['y'].astype(int)
    if debug: print("converting x to list")
    df['x'] = df['x'] = df.loc[:, 'x'].apply(lambda x: [1 if '1' == a else 0 for a in x.split(', ')])
    length = df.shape[0]
    df = df.set_index([range(0,length)])
    return df;

df = unpickle_csv('../data/csv_input/2015-12-07-csv', range(1,21))
fries_ready()
df.head()

../data/csv_input/2015-12-07-csv1
../data/csv_input/2015-12-07-csv2
../data/csv_input/2015-12-07-csv3
../data/csv_input/2015-12-07-csv4
../data/csv_input/2015-12-07-csv5
../data/csv_input/2015-12-07-csv6
../data/csv_input/2015-12-07-csv7
../data/csv_input/2015-12-07-csv8
../data/csv_input/2015-12-07-csv9
../data/csv_input/2015-12-07-csv10
../data/csv_input/2015-12-07-csv11
../data/csv_input/2015-12-07-csv12
../data/csv_input/2015-12-07-csv13
../data/csv_input/2015-12-07-csv14
../data/csv_input/2015-12-07-csv15
../data/csv_input/2015-12-07-csv16
../data/csv_input/2015-12-07-csv17
../data/csv_input/2015-12-07-csv18
../data/csv_input/2015-12-07-csv19
../data/csv_input/2015-12-07-csv20
converting x to list


,Unnamed: 0,move,piece_moved,piece_captured,preboard,postboard,x,y
0,0,"(82, 72)",p,.,"[' RNBKQBNR\n', ' PPP.PPPP\n', ' ........\n', ...","([' RNBKQBNR\n', ' PPP.PPPP\n', ' ........\n',...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",1
1,0,"(84, 74)",p,.,"[' RNBKQBNR\n', ' PPP.PPPP\n', ' ........\n', ...","([' RNBKQBNR\n', ' PPP.PPPP\n', ' ........\n',...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",1
2,0,"(81, 71)",p,.,"[' RNBKQBNR\n', ' PPP.PPPP\n', ' ........\n', ...","([' RNBKQBNR\n', ' PPP.PPPP\n', ' ........\n',...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",1
3,0,"(84, 64)",p,.,"[' RNBKQBNR\n', ' PPP.PPPP\n', ' ........\n', ...","([' RNBKQBNR\n', ' PPP.PPPP\n', ' ........\n',...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",1
4,0,"(92, 71)",n,.,"[' RNBKQBNR\n', ' PPP.PPPP\n', ' ........\n', ...","([' RNBKQBNR\n', ' PPP.PPPP\n', ' ........\n',...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",1


In [65]:
#shuffle
df = df.reindex(np.random.permutation(df.index))

<type 'list'>


In [66]:
df.head()

,Unnamed: 0,move,piece_moved,piece_captured,preboard,postboard,x,y
13287,0,"(82, 73)",k,.,"[' .....R.R\n', ' .P.NK...\n', ' ..P...P.\n', ...","([' .....R.R\n', ' .P.NK...\n', ' ..P...P.\n',...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
150456,0,"(88, 78)",p,.,"[' K...Q..R\n', ' PP.B....\n', ' ..R.P..B\n', ...","([' K...Q..R\n', ' PP.B....\n', ' ..RrP..B\n',...","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...",0
73069,0,"(74, 85)",B,.,"[' ..r..rk.\n', ' .b..qpp.\n', ' pp..pn.p\n', ...","([' ..r..rk.\n', ' .b..qpp.\n', ' pp..pn.p\n',...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
148672,0,"(95, 73)",q,.,"[' .KR..BNR\n', ' PPP...PP\n', ' ....QP..\n', ...","([' .KR..BNR\n', ' .PP...PP\n', ' ....QP..\n',...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
86699,0,"(56, 23)",B,.,"[' ...r.rk.\n', ' pp...p..\n', ' .......p\n', ...","([' ..Br.rk.\n', ' pp...p..\n', ' .......p\n',...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1


In [67]:
all_train = df.iloc[:185000, :]
# all_train = all_train[[s.piece_moved.islower() for s in all_train['move_info']]]
# all_train = all_train[[s.piece_moved.isupper() for s in all_train['move_info']]]
all_test = df.iloc[-1000:, :]
# all_test = all_test[[s.piece_moved.islower() for s in all_test['move_info']]]
#all_test = all_test[[s.piece_moved.isupper() for s in all_test['move_info']]]
X_train = list(all_train['x'])
X_test = list(all_test['x'])
y_train = [[1] if y == 1 else [0] for y in all_train['y']]
y_test = [1 if y == 1 else 0 for y in all_test['y']]

In [73]:
class Experiment:
    #takes in full dataframe and converts to pybrain dataset
    def build_dataset(self, df):
        ds = ClassificationDataSet(1536, nb_classes=2, class_labels=["Illegal", "Legal"])
        for i in df.index:
            ds.addSample(df.loc[i,'x'], df.loc[i,'y'])
        ds._convertToOneOfMany(bounds=[0, 1])
        return ds

    #adds specific piece confusion matrices to results dict d
    def add_piece_specifics(d):
        pass
        
    
    def run_experiment(self, train_all, test_all, num_epochs):
        train_ds = self.build_dataset(train_all)
        test_ds = self.build_dataset(test_all)

        net     = buildNetwork(1536, 100, 2, bias=True, hiddenclass=TanhLayer)
        trainer = BackpropTrainer(net,  train_ds, verbose=True)
        print "Built network"
        y_pred = []

        trainer.trainEpochs(num_epochs)
        prob_out = net.activateOnDataset(test_ds)
        y_pred = prob_out.argmax(axis=1)  # the highest output activation gives the class
        #out = out.reshape(X_train.shape)
        #print(y_pred)
        #print(prob_out)
        cm = confusion_matrix(y_test, y_pred)
        print(cm)
    #    print(confusion_matrix(y_train, out))
        return pd.DataFrame({"train_size": train_all.shape[0], \
                            "train_white_count" : sum([1 if a.isupper() else 0 for a in train_all['piece_moved']]),\
                            "confusion_matrix" : [cm],
                            "accuracy": [(cm[0][0] + cm[1][1]) * 1.0 / (sum([sum(c) for c in cm]))]})

# results_df = pd.DataFrame()
# for train_instances in [10, 500, 1000, 2000, 3000, 4000, 5000]:
#     e = Experiment()
#     results_df = results_df.append(e.run_experiment(all_train[:train_instances], all_test, 1))
e = Experiment()
e.run_experiment(all_train[:5000], all_test, 1)
fries_ready()

Built network


KeyboardInterrupt: 

In [53]:
results_df

,accuracy,confusion_matrix,train_size,train_white_count
0,0.484,"[[199, 291], [225, 285]]",10,4
0,0.500,"[[284, 206], [294, 216]]",500,234
0,0.481,"[[157, 333], [186, 324]]",1000,477
0,0.493,"[[211, 279], [228, 282]]",2000,953
0,0.480,"[[256, 234], [286, 224]]",3000,1456
0,0.495,"[[335, 155], [350, 160]]",4000,1936
0,0.494,"[[273, 217], [289, 221]]",5000,2408


In [30]:
#[1 if a.isupper() else 0 for a in all_train['piece_moved']]
df.head()

,move,piece_moved,piece_captured,preboard,postboard,x,y
9690,"(88, 68)",p,.,"[ RNBKQBNR\n, PPP.PPPP\n, ........\n, ...P....","([ RNBKQBNR\n, PPP.PPPP\n, ........\n, ...P...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",0
402,"(71, 81)",R,.,"[ r.....k.\n, pprn.p.q\n, ..p.bBp.\n, .PPpP...","([ r.....k.\n, pp.n.prq\n, ..p.bBp.\n, .PPp...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
6565,"(86, 85)",q,.,"[ .K.R.BNR\n, PPP.P.PP\n, ........\n, ........","([ .K.R.BNR\n, PPP.P.PP\n, ........\n, .......","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
915,"(94, 95)",R,.,"[ .....rk.\n, .....pp.\n, ..p.q.np\n, .pQ.P...","([ .....rk.\n, .....pp.\n, ..p.q.np\n, .pQ....","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
2743,"(75, 76)",R,.,"[ ..r...k.\n, .p.R..bp\n, r...p.p.\n, p...P...","([ ......k.\n, .p.R..bp\n, r...p.p.\n, p......","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0


In [9]:
#train a simple neural network on 2000 instances
print "Starting"
# build dataset
train_ds = ClassificationDataSet(1536, nb_classes=2, class_labels=["Illegal", "Legal"])
for i in range(0,500):
    #j = random.randint(0,2000)
    train_ds.addSample(X_train[i], y_train[i])
    
test_ds = ClassificationDataSet(1536, nb_classes=2, class_labels=["Illegal", "Legal"])
for i in range(0,500):
    test_ds.addSample(X_test[i], y_test[i])
print "Building dataset"
#print(train_ds.getField('target'))
train_ds._convertToOneOfMany(bounds=[0, 1])
test_ds._convertToOneOfMany(bounds=[0, 1])
#print(train_ds.getField('target'))
print "Converted"
net     = buildNetwork(1536, 100, 2, bias=True, hiddenclass=TanhLayer)
trainer = BackpropTrainer(net,  train_ds, verbose=True)
print "Built network"
y_pred = []
for i in range(1):
    trainer.trainEpochs( 1 )
    prob_out = net.activateOnDataset(test_ds)
    y_pred = prob_out.argmax(axis=1)  # the highest output activation gives the class
    #out = out.reshape(X_train.shape)
    print(y_pred)
    #print(prob_out)
    print(confusion_matrix(y_test, y_pred))
#    print(confusion_matrix(y_train, out))


Starting
Building dataset
Converted
Built network
Total error: 8.2274069558
[1 1 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1 1 0 1 1 1 1 1 1 0 1 0 0 1
 0 0 1 0 0 1 1 1 1 0 0 0 1 0 1 1 1 1 0 1 0 1 0 1 0 1 0 0 0 1 0 0 0 1 0 0 1
 0 0 1 1 1 1 0 1 0 0 0 0 0 0 1 0 1 1 0 1 1 0 0 1 0 1 1 1 1 0 1 0 1 0 0 1 1
 1 1 0 0 1 1 1 0 1 0 1 1 1 0 0 0 1 0 1 0 0 1 1 1 1 1 1 1 0 1 1 1 1 0 0 0 1
 0 1 0 1 0 1 0 1 1 1 1 1 0 1 1 0 0 1 1 0 1 1 0 0 0 1 1 0 0 1 0 1 1 0 1 1 0
 1 1 1 0 0 1 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 1 0 0 0 0 1 0 1 0
 1 0 1 1 0 0 1 1 0 0 1 0 0 1 0 1 1 0 0 0 1 0 0 0 1 0 1 0 1 1 0 0 0 1 1 0 0
 1 1 1 1 1 1 0 1 1 0 1 0 1 1 0 0 0 0 0 0 0 1 0 0 1 1 1 0 0 1 1 1 1 1 0 0 0
 1 0 1 1 0 1 1 0 1 1 1 1 1 0 0 0 1 1 0 0 0 1 0 0 0 0 1 1 0 1 1 1 0 1 1 1 1
 0 0 1 0 1 0 0 0 1 1 0 1 1 1 1 0 0 0 1 1 1 1 0 0 1 0 0 1 1 1 1 1 1 1 0 1 1
 0 0 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 1 0 0 0 1 1 0 1 1 0 1 0 1 1 0 1 1 0 1
 0 1 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 1 0 0 1 0 0 1 1 1 0 0
 0 0 1 0 0 1 1 1 0 0 1 1

In [10]:
y_true = pd.Series(y_test)
y_pred = pd.Series(y_pred)
pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,0,1,All
True,,,
0,125,119,244
1,119,137,256
All,244,256,500


In [11]:
y_pred = pd.Series(y_pred)
length = all_test.shape[0]
all_test = all_test.set_index([range(0,length)])
all_test.loc[:,'predicted'] = y_pred

In [12]:
y_pred.head()

0    1
1    1
2    0
3    0
4    1
dtype: int64

In [13]:
all_test = all_test.iloc[:500,:]

In [15]:
def collect_results(test_df, predicted_y)
    result_df = pd.DataFrame(columns=['training_instances', 'testingpct_white_moves',])
    
    #add predicted
    length = all_test.shape[0]
    test_df = all_test.set_index([range(0,length)])
    test_df.loc[:,'predicted'] = y_pred
    
    #calculate overall confusion matrix
    cm = confusion_matrx(test_df['y'], predicted_y)
    
    #calculate each piece confusion matrix
    for p in "pPrRnNbBqQkK":
        specific_piece = all_test[all_test['piece_moved'] == p]
#         print(p)
        cm = confusion_matrix(specific_piece['y'], specific_piece['predicted'])
        test_df.loc[:, p + '_perf'] = cm
#         print(cm)
#         print(1.0 * cm[0][0] / (sum([sum(a) for a in cm])))

p
[[36 20]
 [29 16]]
0.356435643564
P
[[13 23]
 [16 23]]
0.173333333333
r
[[ 5  7]
 [ 8 10]]
0.166666666667
R
[[14  6]
 [12 12]]
0.318181818182
n
[[17  4]
 [11  6]]
0.447368421053
N
[[ 4 11]
 [ 4 12]]
0.129032258065
b
[[6 3]
 [8 8]]
0.24
B
[[ 5 11]
 [ 2 16]]
0.147058823529
q
[[11 10]
 [ 6 11]]
0.289473684211
Q
[[ 9  8]
 [ 7 10]]
0.264705882353
k
[[2 9]
 [8 6]]
0.08
K
[[3 7]
 [8 7]]
0.12
